# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import urllib.parse as up

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,28.98,87,64,2.24,EC,1689287432
1,1,weno,7.4515,151.8468,28.58,75,69,5.46,FM,1689287203
2,2,muzaffargarh,30.0754,71.1921,33.01,34,0,5.04,PK,1689287485
3,3,albany,42.6001,-73.9662,28.43,94,68,1.04,US,1689287731
4,4,adamstown,-25.0660,-130.1015,21.16,77,1,2.75,PN,1689287794


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

plot_settings = {'x':city_data_df['Lng'], 'y':city_data_df['Lat'], 'kind':'points', 'color':city_data_df['Cloudiness'], 'alpha':0.5, 'tiles':'ESRI', 'size':city_data_df['Humidity']}


# Display the map
    
# city_data_df.hvplot(**plot_settings)

city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='Cloudiness', alpha=0.5, tiles='ESRI', size='Humidity')


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Cloudiness,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_df_narrowed_temp = city_data_df.dropna(subset='Max Temp')[city_data_df['Max Temp'] < 30]

city_df_narrowed_temp_humidity = city_df_narrowed_temp.dropna(subset='Humidity')[city_df_narrowed_temp['Humidity'] > 40]

# Display sample data
city_df_narrowed_temp_humidity.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,28.98,87,64,2.24,EC,1689287432
1,1,weno,7.4515,151.8468,28.58,75,69,5.46,FM,1689287203
3,3,albany,42.6001,-73.9662,28.43,94,68,1.04,US,1689287731
4,4,adamstown,-25.0660,-130.1015,21.16,77,1,2.75,PN,1689287794
5,5,new norfolk,-42.7826,147.0587,14.95,84,94,3.88,AU,1689287794


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_df_narrowed_temp_humidity.copy()[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,87,
1,weno,FM,7.4515,151.8468,75,
3,albany,US,42.6001,-73.9662,94,
4,adamstown,PN,-25.0660,-130.1015,77,
5,new norfolk,AU,-42.7826,147.0587,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'apiKey':geoapify_key, 'limit':1, 'categories':'accommodation.hotel'}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
    

    # Make an API request using the params dictionary
    request_url = base_url + up.urlencode(params)
    
    name_address = requests.get(request_url).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
weno - nearest hotel: High Tide Hotel
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
whangarei - nearest hotel: Grand Hotel
troitskoye - nearest hotel: Березка
klyuchi - nearest hotel: No hotel found
paracuru - nearest hotel: Vento Brasil
nadi - nearest hotel: President Hotel
bredasdorp - nearest hotel: Victoria Hotel
grytviken - nearest hotel: No hotel found
abu zabad - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
utrik - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
mossel bay - nearest hotel: Protea Hotel Mossel Bay
ribeira grande - nearest hotel: Hospedaria JSF
queenstown - nearest hotel: Queens Hotel
saldanha - nearest hotel: Hoedjesbaai
blackmans bay - nearest hotel: Villa Howden
zlin

malango - nearest hotel: Madarana
kuching - nearest hotel: Hotel Samudera
puerto baquerizo moreno - nearest hotel: Dory's House
auki - nearest hotel: No hotel found
cadale - nearest hotel: No hotel found
sumbawa besar - nearest hotel: Hotel Mekarsari
paradip garh - nearest hotel: Hotel Sea Pearl
lent'ekhi - nearest hotel: No hotel found
yumaguzino - nearest hotel: No hotel found
saint-denis - nearest hotel: Residhome
margate - nearest hotel: The Reading Rooms
sao felix do xingu - nearest hotel: Hotel Terraço
tolanaro - nearest hotel: Hôtel Mahavokey
nar'yan-mar - nearest hotel: Гостиница "Агат"
kamensk-ural'skiy - nearest hotel: Октябрь
saint-joseph - nearest hotel: PALM Hotel & Spa
katiola - nearest hotel: Hôtel Amitie
bungo-takada-shi - nearest hotel: AZ Hotel
ulsteinvik - nearest hotel: Quality Hotel Ulstein
polyarnyy - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
chonchi - nearest hotel: Hotel Huildín
jaru - nearest hotel: Hotel avenida
chimbarongo - ne

salinopolis - nearest hotel: Hotel Salinópolis
geraldton - nearest hotel: Ocean Centre Hotel
batticaloa - nearest hotel: Lake View Hotel
eldridge - nearest hotel: No hotel found
carbonear - nearest hotel: No hotel found
zuunmod - nearest hotel: New World Hotel
oppdal - nearest hotel: Quality Hotel Skifer
akhalk'alak'i - nearest hotel: Hotel Gold
betio village - nearest hotel: Seaman Hostel
hulan ergi - nearest hotel: China First Heavy Industries Hotel
ban houakhoua - nearest hotel: Monethong Hotel
vilyuysk - nearest hotel: Мария
naberera - nearest hotel: No hotel found
ivdel' - nearest hotel: гостиница Ивдель
inirida - nearest hotel: Residencias El Turista del Guainia
nova borova - nearest hotel: Готель "Титан"
fort-de-france - nearest hotel: Simon Hôtel
xinzhi - nearest hotel: No hotel found
rouached - nearest hotel: No hotel found
panacan - nearest hotel: Gorayan Lodge
ust-nera - nearest hotel: гостиница "Солнечная"
jalai nur - nearest hotel: 满洲里福门客栈
pangody - nearest hotel: Газпром 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,87,Hostal La Mirada De Solitario George
1,weno,FM,7.4515,151.8468,75,High Tide Hotel
3,albany,US,42.6001,-73.9662,94,No hotel found
4,adamstown,PN,-25.0660,-130.1015,77,No hotel found
5,new norfolk,AU,-42.7826,147.0587,84,Woodbridge on the Derwent
...,...,...,...,...,...,...
574,ponta delgada,PT,37.7333,-25.6667,83,Hotel Talisman
575,ves'yegonsk,RU,58.6640,37.2604,99,Гостиница Весь
577,waingapu,ID,-9.6567,120.2641,85,HOTEL SANDLEWOOD
578,ancud,CL,-41.8697,-73.8203,95,Cabañas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot

city_hotel_df = city_df_narrowed_temp_humidity.merge(hotel_df,'left')

# Display the map

city_hotel_df.hvplot.points('Lng', 'Lat', geo=True, color='Cloudiness', alpha=0.5, tiles='ESRI', size='Humidity', hover_cols=['Hotel Name','Country'])


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Cloudiness,Humidity,Hotel Name,Country)